## Trajectory linking 

In [27]:
import tobac 
import glob 
import numpy as np
import pandas as pd 
import os 
                                                                                                        
## Recombination of feature dataframes (update framenumbers)                                            
                                                                                                        
savedir= '/media/juli/Data/projects/data/satellite_data/ncep/ctt/Save/tbbtracking'                      
                                                                                                        
# read in HDF5 files with saved features                                                                
file_list= glob.glob(savedir  + '/Features_cells_2000??.h5')                                            
file_list.sort()                                                                                        
print('nr. of monthly feature files:', len(file_list))                                                  
                                                                                                        
                                                                                                       
i = 0                                                                                                   
frames = 0         

nr. of monthly feature files: 11


In [26]:
# example input 

file = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_200002.nc4'
ds= Dataset(file)
tbb = np.array(ds['Tb']) 

In [28]:
from netCDF4 import Dataset

for file in file_list:                                                                                  
    if i == 0:                                                                                          
        Features = pd.read_hdf(file, 'table')                                                           
        # read in data mask with segments for tracked cells                                             
        date= file[len(file)-9: len(file)-3]                                                            
        ds = Dataset(savedir+ '/Mask_Segmentation_'+date+'.nc')                                   
        mask = np.array(ds['segmentation_mask'])                                                        
        # update total nr of frames                                                                     
        frames += np.shape(mask)[0] -1                                                                  
        i = 1                                                                                           
        print('file for: ',date, 'rows: ',Features.shape[0], 'frames: ', frames)                        
                                                                                                        
    features = pd.read_hdf(file, 'table')                                                               
    # update frame number and make sure they are sequential                                             
    features['frame_new'] = features['frame']  + frames                                                     
    # append dataframes                                                                                 
    Features = Features.append(features, ignore_index=True)                                             
    # read in data mask with segments for tracked cells                                                 
    date= file[len(file)-9: len(file)-3]                                                                
    ds = Dataset(savedir+ '/Mask_Segmentation_'+date+'.nc')                                       
    mask = np.array(ds['segmentation_mask'])                                                            
    #update total nr of frames                                                                          
    frames += np.shape(mask)[0]                                                                         
    print('file for: ',date, 'rows: ',features.shape[0], 'frames: ', frames)                            
                                                                                                                                                     

file for:  200002 rows:  2655 frames:  1057


/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


file for:  200002 rows:  2655 frames:  2115
file for:  200003 rows:  6110 frames:  3603
file for:  200004 rows:  9489 frames:  5043
file for:  200005 rows:  16083 frames:  6531
file for:  200006 rows:  23183 frames:  7971
file for:  200007 rows:  28797 frames:  9459
file for:  200008 rows:  25743 frames:  10947
file for:  200009 rows:  13539 frames:  12387
file for:  200010 rows:  10393 frames:  13875
file for:  200011 rows:  5105 frames:  15315
file for:  200012 rows:  6118 frames:  16803


In [43]:
## Tracking                                                                                             
# Dictionary containing keyword arguments for the linking step:                                         
parameters_linking={}                                                                                   
parameters_linking['adaptive_stop']=0.2                                                                 
parameters_linking['adaptive_step']=0.95                                                                
parameters_linking['extrapolate']=0                                                                     
parameters_linking['order']=1                                                                           
parameters_linking['subnetwork_size']= 5000 # maximum size of subnetwork used for linking               
parameters_linking['memory']= 0                                                                          
parameters_linking['time_cell_min']= 6*dt                                                              
parameters_linking['method_linking']='predict'                                                          
#parameters_linking['method_detection']='threshold'                                                     
parameters_linking['v_max']= 10                                                                         
#parameters_linking['d_min']=2000                                                                       
parameters_linking['d_min']=4*dxy # four times the grid spacing ?                                       
            

In [44]:
## Perform trjactory linking with trackpy   
dt = 1800
dxy = 14126.0
Track=tobac.linking_trackpy(Features,tbb, dt=dt,dxy=dxy,**parameters_linking)                         
Track.to_hdf(os.path.join(savedir,'Tracks_tbb_2000.h5'),'table')                                          
                                                                                                        
                      

Frame 16802: 3 trajectories present.


/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['time', 'timestr']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [33]:
help(tobac.linking_trackpy)

Help on function linking_trackpy in module tobac.tracking:

linking_trackpy(features, field_in, dt, dxy, v_max=None, d_max=None, d_min=None, subnetwork_size=None, memory=0, stubs=1, time_cell_min=None, order=1, extrapolate=0, method_linking='random', adaptive_step=None, adaptive_stop=None, cell_number_start=1)
    Function to perform the linking of features in trajectories
    
    Parameters:
    features:     pandas.DataFrame 
                  Detected features to be linked             
    v_max:        float
                  speed at which features are allowed to move
    dt:           float
                  time resolution of tracked features
    dxy:          float
                  grid spacing of input data
    memory        int
                  number of output timesteps features allowed to vanish for to be still considered tracked
    subnetwork_size int
                    maximim size of subnetwork for linking  
    method_detection: str('trackpy' or 'threshold')
      

In [40]:
Features[Features.threshold_value == 190]

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,ncells
13498,4065,1,8.102885,431.422366,105,190,2079,2000-04-10 15:00:00,2000-04-10 15:00:00,15.760289,103.192237,2029
13502,4066,1,12.123467,429.595127,181,190,2083,2000-04-10 15:30:00,2000-04-10 15:30:00,16.162346,103.009513,1699
13508,4067,1,11.534579,429.695659,123,190,2089,2000-04-10 16:00:00,2000-04-10 16:00:00,16.103457,103.019567,1616
27055,5630,1,6.994782,283.411428,116,190,6147,2000-05-13 05:30:00,2000-05-13 05:30:00,15.649478,88.391141,1172
40060,6714,1,8.882727,210.208494,120,190,3069,2000-06-04 19:30:00,2000-06-04 19:30:00,15.838273,81.070852,751
40799,6754,2,44.395025,199.574633,110,190,3808,2000-06-05 15:30:00,2000-06-05 15:30:00,19.389503,80.007464,819
40819,6755,1,43.255974,197.185342,140,190,3828,2000-06-05 16:00:00,2000-06-05 16:00:00,19.275597,79.768534,1678
56909,7747,2,23.677624,264.697587,129,190,19918,2000-06-26 08:00:00,2000-06-26 08:00:00,17.317763,86.519760,1921
64041,8188,4,42.787403,589.744324,126,190,3867,2000-07-05 12:30:00,2000-07-05 12:30:00,19.228740,119.024434,1311
64099,8190,2,44.995603,587.088751,143,190,3925,2000-07-05 13:30:00,2000-07-05 13:30:00,19.449561,118.758875,1406


In [36]:
Tracks = tracks[tracks.cell >= 0]

print(np.unique(tracks.cell.values).shape[0], '  unique values in year 2000. Features:', tracks.shape[0])

92669   unique values in year 2000. Features: 100645


## Cold core criteria 

In [7]:
# once during the lifetime a cold core of <== 200 K needs to be present 

for i in np.unique(tracks.cell.values):
    subset = tracks[tracks.cell == i]
    if subset[subset.threshold_value <=200].shape[0] == 0:
        tracks.drop(tracks.loc[tracks['cell']== i].index, inplace=True)
 

NameError: name 'track' is not defined

In [8]:

tracks.to_hdf(os.path.join(savedir,'Tracks_tbb_2000_cold_core.h5'),'table')  

/home/juli/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['time', 'timestr']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [6]:
# get tracks with cold core 

tracks = pd.read_hdf(savedir + '/Tracks_tbb_2000.h5', 'table')


tracks.shape

(149870, 14)

**Notes:**
    
- filter for heavy precip
- plot a track step by step, to look at what is being tracked 


## Application of heavy rain core filter

In [52]:
#tracks.sort_values(by = 'cell')

In [63]:

# loop through cells 
for cell in np.unique(tracks.cell.values):
    subset = tracks[tracks.cell == cell]
    print(subset.shape)
    precipitation_flag = 0 
    # loop through timesteps of features for specific cell 
    for f in subset.frame.values: 
        # idx is the timestep index for respective timestep or mask file 
        idx = Features[Features.frame_new == f].frame.values[0] 
    
        # open corresponding precip and mask file 
        year = subset.time.values[0].year 
        month = subset.time.values[0].month
        
        maskfile = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/'+str(year)+'/merg_'+str(year) + str(month) + '.nc4'
        #precip = '/media/juli/Data/Elements/gpm_v06/'+str(year)+'/gpm_'+ str(year)+str(month)+'.nc4'
        #mask = xr.open_datarray(maskfile)
        # precipitation_flag +=1 
        
        print(year, month, maskfile, idx )
        
        
        # check whether precip is in area of segmentation mask, where segmentation mask== feature number 
    

    # remove cell from track dataframe if no precipitation feature is found
    # if precipitation_flag > 0: 
    
    

(8, 14)
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1315
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1316
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1317
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1318
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1319
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1320
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1321
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1322
(9, 14)
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1388
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1389
2000 3 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20003.nc4 1390
2000 3 /media/juli/Data/pro

2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1031
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1032
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1033
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1034
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1035
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1036
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1037
(12, 14)
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1083
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1084
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1085
2000 4 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20004.nc4 1086
2000 4 /media/juli/Data/projects/d

2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 436
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 437
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 438
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 439
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 440
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 441
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 442
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 443
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 444
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 445
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 446
2000 5 /media/juli/Data/projects/data/satellite_data/n

2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 809
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 810
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 811
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 812
(9, 14)
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 804
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 805
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 806
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 807
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 808
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 809
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 810
2000 5 /media/juli/Data/projects/data/satellit

2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1077
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1078
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1079
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1080
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1081
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1082
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1083
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1084
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1085
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1086
2000 5 /media/juli/Data/projects/data/satellite_data/ncep/ctt/2000/merg_20005.nc4 1087
2000 5 /media/juli/Data/projects/data/satel

...

(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


KeyboardInterrupt: 